In [1]:
import os
import numpy as np
import pandas as pd
import warnings
import func
import fnmatch
from params import dresden_images_root, train_csv_path, patches_root, \
    patches_db_path, weights_path, ins_patches_db, ins_weights, \
    ins_train_csv, ins_test_csv, ins_train_path, ins_test_path

In [2]:
warnings.filterwarnings("ignore")

if not os.path.exists(patches_root):
    os.makedirs(patches_root)

In [3]:
# split images dataset for training
images_db = pd.read_csv(train_csv_path)

model_list = np.unique(images_db['brand_model'])
img_list = images_db['path']

if not os.path.exists(patches_db_path):
    train_list, val_list, test_list, info, weights = func.split(img_list, model_list, patches_db_path)
else:
    patches_db = np.load(patches_db_path, allow_pickle=True).item()
    train_list = patches_db['train']
    val_list = patches_db['val']
    test_list = patches_db['test']
    info, weights = func.split_info(train_list, val_list, test_list, 
                                       model_list, total=len(img_list))

# store class weight to csv
df = pd.DataFrame([weights], columns=range(len(model_list)))
df.to_csv(weights_path, index=False)

Canon_Ixus70 in training set: 373.
Canon_Ixus70 in validation set: 90.
Canon_Ixus70 in test set: 104.

Nikon_D200 in training set: 483.
Nikon_D200 in validation set: 127.
Nikon_D200 in test set: 142.

Olympus_mju-1050SW in training set: 655.
Olympus_mju-1050SW in validation set: 160.
Olympus_mju-1050SW in test set: 225.



In [5]:
# split images into patches
for i in range(len(model_list)):
    func.patch(model_list[i], info[i][0], 'train')
    func.patch(model_list[i], info[i][1], 'val')
    func.patch(model_list[i], info[i][2], 'test')

100%|██████████| 225/225 [00:00<00:00, 486954.80it/s]


In [6]:
# split dataset for instance experiment
ins_db = pd.read_csv(ins_train_csv)
img_list = ins_db['path']

if not os.path.exists(ins_patches_db):
    train_list, val_list, test_list, info, weights = func.split(img_list, model_list, ins_patches_db)
else:
    patches_db = np.load(ins_patches_db, allow_pickle=True).item()
    train_list = patches_db['train']
    val_list = patches_db['val']
    test_list = patches_db['test']
    info, weights = func.split_info(train_list, val_list, test_list, 
                                       model_list, total=len(img_list))
    
# get information about the instance test dataset
test_db = pd.read_csv(ins_test_csv)
test_info = []
for m in model_list:
    tmp = fnmatch.filter(test_db['path'].values, m + '*')
    test_info.append(tmp)
    print('Instance experiment for camera {} has {} images.'.format(m, len(tmp)))

# store class weight to csv
df = pd.DataFrame([weights], columns=range(len(model_list)))
df.to_csv(ins_weights, index=False)

Canon_Ixus70 in training set: 114.
Canon_Ixus70 in validation set: 35.
Canon_Ixus70 in test set: 38.

Nikon_D200 in training set: 255.
Nikon_D200 in validation set: 50.
Nikon_D200 in test set: 67.

Olympus_mju-1050SW in training set: 120.
Olympus_mju-1050SW in validation set: 37.
Olympus_mju-1050SW in test set: 47.

Instance experiment for camera Canon_Ixus70 has 380 images.
Instance experiment for camera Nikon_D200 has 380 images.
Instance experiment for camera Olympus_mju-1050SW has 836 images.


In [7]:
for i in range(len(model_list)):
    func.patch(model_list[i], info[i][0], 'train', patches_root=ins_train_path)
    func.patch(model_list[i], info[i][1], 'val' , patches_root=ins_train_path)
    func.patch(model_list[i], info[i][2], 'test', patches_root=ins_train_path)
    func.patch(model_list[i], test_info[i], '.', patches_root=ins_test_path)

100%|██████████| 836/836 [00:00<00:00, 1222181.30it/s]
